In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from functions import *

# Data Exploration (and Cleaning)

In [2]:
url1 = r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\raw\df_final_demo.txt'
url2 = r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\raw\df_final_experiment_clients.txt'
url3 = r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\raw\df_final_web_data_pt_1.txt'
url4 = r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\raw\df_final_web_data_pt_2.txt'

df_final_demo = pd.read_csv(url1)
df_final_experiment_clients = pd.read_csv(url2)
df_web_data_1 = pd.read_csv(url3)
df_web_data_2 = pd.read_csv(url4)

### Check df_final_experiment_clients

In [1]:
data_summary(df_final_experiment_clients)

NameError: name 'data_summary' is not defined

In [4]:
# Create list with client_id for compare each df

list_id_final_demo = df_final_demo['client_id']

list_id_final_exp = df_final_experiment_clients['client_id']

common_clients_id = set(list_id_final_demo).intersection(set(list_id_final_exp))
demo_clients_id = set(list_id_final_demo).difference(set(list_id_final_exp))
exp_clients_id = set(list_id_final_exp).difference(set(list_id_final_demo))

{
    "Commons ID": len(common_clients_id),
    "Demo ID": len(demo_clients_id),
    "Exp ID": len(exp_clients_id)
}

{'Commons ID': 70609, 'Demo ID': 0, 'Exp ID': 0}

After verifying that the NaNs are real customers, we apply a PCA on another file to create a clean dataframe

In [5]:
url5 = r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\datadf_experiment_non_nan.csv'

df_final_experiment_cleaned = pd.read_csv(url5)

In [6]:
df_final_experiment_cleaned

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,Test
70605,8788427,Test
70606,266828,Test
70607,1266421,Test


### Check df_final_demo

In [8]:
df_final_demo.sample(10)
df_final_demo.shape

(70609, 9)